10

In [56]:
from geopy.geocoders import Nominatim
import folium

location_list = ['Bucktown, Chicago',
                 'Boystown, Chicago',
                 'Lake View, Chicago',
                 'Logan Square, Chicago',
                 'Wicker Park, Chicago',
                 'Lincoln Park, Chicago',
                 'West Town, Chicago,',
                 'Uptown, Chicago',
                 'Near North Side, Chicago',
                 'River West, Chicago']

for x, address in enumerate(location_list):
    geolocator = Nominatim(user_agent="chi_explorer")
    location = geolocator.geocode(address)
    location_list[x] = [address.split(',')[0], location.latitude, location.longitude]
location_list

[['Bucktown', 41.9124994, -87.6724672],
 ['Boystown', 41.9438833, -87.6492669],
 ['Lake View', 41.947050000000004, -87.65542878290054],
 ['Logan Square', 41.9285683, -87.7067931],
 ['Wicker Park', 41.9078018, -87.6781551],
 ['Lincoln Park', 41.940297650000005, -87.63811710541756],
 ['West Town', 41.9014213, -87.6861661],
 ['Uptown', 41.9666299, -87.6555458],
 ['Near North Side', 41.9000327, -87.6344975],
 ['River West', 41.8755616, -87.6244212]]

In [25]:
CLIENT_ID = '4N2OKUVXD1N1RQYQJOCBRGF1GBGOML5KLC4WV1GNX0JH3U3S' 
CLIENT_SECRET = 'YZSH4L5VMESUMFUXNAP3AIZ10HZPZMKLV05QRGKA40I3JCVX' 
VERSION = '20180605'

In [78]:
import requests
def getFoodVenues(location_list, radius=805,limit=50):  
    food_list=[]
    for name, l1, l2 in location_list: 
        # create the API request URL
        for offset in [0,50,100]:
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section=food&limit={}&offset={}'.format(
                CLIENT_ID,CLIENT_SECRET,VERSION, 
                l1,l2, 
                radius, 
                limit,offset)  
            # make the GET request
            results=requests.get(url)
            #print(results)
            results=results.json()["response"]['groups'][0]['items']
            #print(name, len(results))
            for item in results:
                food_list.append([name,l1,l2, 
                item['venue']['name'], 
                item['venue']['id'],
                item['venue']['location']['lat'], 
                item['venue']['location']['lng'],  
                item['venue']['categories'][0]['shortName']])
            if len(results) < 50:
                break

    venue_list=pd.DataFrame(food_list)
    venue_list.columns=['Location','Latitude','Longitude', 
                  'Business','id',
                  'Business Latitude','Business Longitude','Business Category']
    
    return venue_list

In [79]:
df_food_bus = getFoodVenues(location_list)
df_food_bus

,Location,Latitude,Longitude,Business,id,Business Latitude,Business Longitude,Business Category
0,Bucktown,41.912499,-87.672467,Club Lucky,4b9ea8c3f964a52076f736e3,41.912457,-87.673714,Italian
1,Bucktown,41.912499,-87.672467,Las Palmas,46b0836bf964a5208c491fe3,41.910218,-87.673866,Mexican
2,Bucktown,41.912499,-87.672467,etta,5b4804edc0af57002c6bf071,41.910590,-87.674389,New American
3,Bucktown,41.912499,-87.672467,Piece Brewery and Pizzeria,40fc6080f964a5202d0b1fe3,41.910403,-87.676145,Pizza
4,Bucktown,41.912499,-87.672467,Lillie's Q,4c4e3cfa92b6a593686f8f6f,41.910592,-87.674988,BBQ
5,Bucktown,41.912499,-87.672467,Amaru,5d268de2681ae90023aeb5fd,41.910715,-87.675373,Latin American
6,Bucktown,41.912499,-87.672467,Paradise Park,5b78d5df0802d4002c8bb6ea,41.910411,-87.675669,Pizza
7,Bucktown,41.912499,-87.672467,Bonci Pizzeria,5ada9afe41868621d2cce37b,41.910194,-87.677409,Pizza
8,Bucktown,41.912499,-87.672467,Dimo's Pizza,51eee947498e5566f01655c0,41.910909,-87.677398,Pizza
9,Bucktown,41.912499,-87.672467,Stan's Donuts & Coffee,52f2eebd498ea23d1a8d16d5,41.909927,-87.677462,Donuts
